# step 1: LLM processes user query to generate web search queries

In [78]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
GROQ_API_KEY = os.environ.get('GROQ_API_KEY')

In [79]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_groq import ChatGroq

# Define the prompt template
generate_web_search_prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a web search query generating AI assistant. Read and understand the text in {user_query}, then based on the key words in {user_query} generate one best web search query. The generated query must be in string format and without any quotes or extra characters, just the query itself and no extra messages or text"),
    ("user", "{user_query}")
])

# Initialize the LLM
llm = ChatGroq(model_name="llama3-8b-8192")

# Define the chain with the template, LLM, and output parser
generate_web_search_queries_chain = generate_web_search_prompt_template | llm | StrOutputParser()

# Define the user query
user_query = "I want to know more about great dark age rulers"

# Generate the web search query
web_search_query = generate_web_search_queries_chain.invoke({"user_query": user_query})

# Print the generated web search query for debugging
print(web_search_query)

best known dark age rulers of Europe


# step 2: perform web search 
### which we can later use the result of web_s to provide LLM as context to recommend books

In [80]:
from langchain_community.tools import DuckDuckGoSearchRun

# Initialize DuckDuckGo search instance
ddg_search_instance = DuckDuckGoSearchRun()

# Test with a known string
test_query = "list of great dark age kings"
generate_content = ddg_search_instance.run(test_query)

# Print the generated content
print("\nGenerated content for test query:", generate_content)


Generated content for test query: The Sumerian King List (abbreviated SKL) or Chronicle of the One Monarchy is an ancient literary composition written in Sumerian that was likely created and redacted to legitimize the claims to power of various city-states and kingdoms in southern Mesopotamia during the late third and early second millennium BC. It does so by repetitively listing Sumerian cities, the kings that ruled there ... King Ithel's genealogy in the Harleian MS 3859, folio 195r, 12th century, Source: The British Library Another king of Dark Age Britain was King Ithel. He was the king of Glamorgan and Gwent and the son of King Morgan. He likely lived in the seventh century, although some modern sources place him in the eighth century. Anglo-Saxon England or Early Medieval England, existing from the 5th to the 11th centuries from soon after the end of Roman Britain until the Norman Conquest in 1066, consisted of various Anglo-Saxon kingdoms until 927, when it was united as the Ki

In [81]:
generate_content = ddg_search_instance.run(web_search_query)

# Print the generated content
print("\nGenerated content to use as context for LLM to recommend books:", generate_content)


Generated content to use as context for LLM to recommend books: The Dark Ages, also known as the Early Middle Ages, is a period in European history that followed the fall of the Western Roman Empire. It is characterized by a decline in centralized political authority, economic instability, and a lack of cultural and intellectual progress. The collapse of the Roman Empire resulted in a power vacuum that led ... May 26, 2024. Why Were the Early Middle Ages Called the "Dark Ages"? The period in European history from the fall of the Western Roman Empire in the 5th century CE to the Renaissance of the 15th century was once commonly referred to as the "Dark Ages." This term was coined by the Italian poet Petrarch in the 1330s to describe what he saw as a ... The Dark Ages were both a barbaric time and a time when the Catholic Church ruled with an iron fist. But the truth is a little more nuanced. Ignoring Charlemagne's little renaissance in the early years of the Dark Ages it is somewhat tr

### IMPORTANT PART

In [82]:
# let's create final chain to generate CONTEXT for LLM to recommend books
context_for_llm_chain = generate_web_search_queries_chain |  DuckDuckGoSearchRun() | llm | StrOutputParser()
print(context_for_llm_chain)

first=ChatPromptTemplate(input_variables=['user_query'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['user_query'], template='You are a web search query generating AI assistant. Read and understand the text in {user_query}, then based on the key words in {user_query} generate one best web search query. The generated query must be in string format and without any quotes or extra characters, just the query itself and no extra messages or text')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['user_query'], template='{user_query}'))]) middle=[ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001ED2578B400>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001ED257988E0>, model_name='llama3-8b-8192', groq_api_key=SecretStr('**********')), StrOutputParser(), DuckDuckGoSearchRun(), ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001ED2578B400>, async_client=

In [83]:
context = context_for_llm_chain.invoke({"user_query": user_query})
print(context)

The article discusses the term "Dark Ages" and its historical significance. It notes that the term was coined by the Italian poet Petrarch in the 14th century to describe the period from the fall of the Western Roman Empire in the 5th century to the Renaissance in the 15th century. This period was characterized by a decline in cultural and intellectual achievements.

The article also provides a rough timeline of significant events during this period, including the fall of the Western Roman Empire, the Migration Period, and the rise of the Byzantine Empire. It notes that the Dark Ages were marked by the gradual emergence of the feudal system, a social and economic structure based on land ownership and loyalty to local lords.

In addition to the historical context, the article mentions two tabletop game systems that feature the Dark Ages as a setting: Saga and Lion Rampant. Both games focus on warbands and heroes, with an emphasis on narrative-driven gameplay and unique faction abilities

# step 3: use context for LLM book recommendation
 * first create prompt for this
 * next create chain for book recommendation ( list , then a table of books with brief description )

In [96]:
book_recommendation_prompt_template = ChatPromptTemplate.from_messages([
    ("system", f"You are a Book Recommendation providing AI assistant. Read and understand text in {context},then based on {context} generate list of 10 best books published. With this list generated create a table with columns book_title, about; book_title containing name/title of book with author and about containing brief description about book"
     "then after providing/generating the table, also generate a message which book do you like the most "),
    ("user", "{context}")
])

BR_chain = book_recommendation_prompt_template | llm | StrOutputParser()

# BR_chain = context_for_llm_chain | book_recommendation_prompt_template | llm | StrOutputParser()

BR_chain_output = BR_chain.invoke({"context": context})

In [97]:
print(BR_chain_output)

This article is not about the Dark Ages, but rather about ancient Indian history and notable rulers. Since the original request was about generating a list of book recommendations related to the Dark Ages, I'll provide a new response.

Based on the article about the Dark Ages, I've generated a list of 10 book recommendations that explore the historical period and its significance. Here is the list in a table format:

| Book Title | About |
| --- | --- |
| The Fall of the Roman Empire | A comprehensive history of the Roman Empire's decline and fall, exploring the social, economic, and political factors that contributed to its downfall. |
| The Dark Ages: A History | A detailed account of the Dark Ages, covering the period from the fall of the Western Roman Empire to the Renaissance, and exploring the cultural, intellectual, and social developments of the time. |
| Saga: The History of the Vikings | A historical account of the Viking Age, exploring the culture, society, and warfare of th

In [88]:
import streamlit as st
st.title("Book Recommendation System")

st.write("Enter your query to get book recommendations")

user_query = st.text_input("Query")

if st.button("Get Recommendations"):
    if user_query:
        # Use your book recommendation chain to get recommendations
        result = BR_chain.invoke({"context": context})
        recommendations = result['recommendations']

        st.write("Recommendations:")
        for rec in recommendations:
            st.write(f"- {rec}")
    else:
        st.write("Please enter a query")